In [1]:
import sys
sys.path.append('../implementation/')
import ast
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.special as sp
from tqdm import tqdm
import time
from healey_adaboost_naive_bayes import AdaBoostNB
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Loading the STL Crimes underlying data and user interaction data
underlying_data = pd.read_csv('../data/stl_crimes/dots.csv')
underlying_data.set_index('id', drop=True, inplace=True)

# this is the main trick; not sure why, but categories should be in the range [0, n-1], not [1, n]
underlying_data['type'] = underlying_data['type'] - 1
output_file_path = '../output/stl/stl_map_results_ada_nb.pkl'

interaction_data = pd.read_csv('../data/stl_crimes/stl_combined_interactions.csv')
interaction_data['interaction_session'] = interaction_data.apply(lambda row: ast.literal_eval(row.interaction_session), axis=1)
interaction_data['interaction_type_session'] = interaction_data.apply(lambda row: ast.literal_eval(row.interaction_type_session), axis=1)
ks = [1, 5, 10, 20, 50, 100]

In [3]:
stl_map_results = pd.DataFrame()

for participant_index, row in interaction_data.iterrows():
    print(f'Processing user {row.user} task {row.task}')
    results = {'participant_id': row.user, 'task': row.task}
    model = AdaBoostNB(underlying_data, [['x','y']], ['type'])
    predicted = pd.DataFrame()
    rank_predicted = []
    for i in tqdm(range(len(interaction_data.iloc[participant_index].interaction_session))):
        interaction = interaction_data.iloc[participant_index].interaction_session[i]
        model.update(interaction)

        if i < len(interaction_data.iloc[participant_index].interaction_session) - 1:
            probability_of_next_point = model.predict()
            next_point = interaction_data.iloc[participant_index].interaction_session[i+1]
            predicted_next_dict = {}
            for k in ks:
                predicted_next_dict[k] = (next_point in probability_of_next_point.nlargest(k).index.values)
            predicted = predicted.append(predicted_next_dict, ignore_index=True)
            sorted_prob = probability_of_next_point.sort_values(ascending=False)
            rank, = np.where(sorted_prob.index.values == next_point)
            rank_predicted.append(rank[0] + 1)
            
    ncp = predicted.sum()/len(predicted)
    for col in ncp.index:
        results[f'ncp-{col}'] = ncp[col]
    results['rank'] = rank_predicted    
    stl_map_results = stl_map_results.append(results, ignore_index=True)
    
stl_map_results.to_pickle(output_file_path)

Processing user 15 task geo-based


100%|██████████| 54/54 [00:04<00:00, 11.00it/s]


Processing user 14 task geo-based


100%|██████████| 46/46 [00:04<00:00,  9.77it/s]


Processing user 28 task geo-based


...

100%|██████████| 21/21 [00:01<00:00, 11.65it/s]


Processing user 19 task type-based


100%|██████████| 11/11 [00:00<00:00, 11.68it/s]


Processing user 18 task type-based


100%|██████████| 12/12 [00:01<00:00, 11.56it/s]


In [11]:
stl_map_results

,participant_id,task,ncp-1,ncp-5,ncp-10,ncp-20,ncp-50,ncp-100,rank
0,15,geo-based,0.0,0.037736,0.150943,0.188679,0.245283,0.471698,"[105, 166, 32, 225, 10, 11, 35, 28, 11, 7, 10,..."
1,14,geo-based,0.0,0.000000,0.044444,0.177778,0.288889,0.400000,"[7, 311, 20, 15, 13, 11, 188, 27, 11, 128, 31,..."
2,28,geo-based,0.0,0.021739,0.043478,0.217391,0.369565,0.586957,"[4, 89, 10, 15, 14, 182, 21, 1, 50, 56, 57, 51..."
3,16,geo-based,0.0,0.046512,0.093023,0.093023,0.186047,0.395349,"[37, 4, 491, 31, 23, 9, 41, 37, 10, 230, 48, 4..."
4,17,geo-based,0.0,0.020408,0.061224,0.102041,0.306122,0.489796,"[61, 249, 184, 0, 26, 4, 1, 154, 54, 48, 48, 2..."
...,...,...,...,...,...,...,...,...,...
73,21,type-based,0.0,0.166667,0.583333,0.916667,0.916667,1.000000,"[7, 53, 3, 9, 4, 13, 7, 9, 8, 10, 13, 12]"
74,23,type-based,0.0,0.333333,0.733333,0.866667,0.933333,1.000000,"[3, 85, 3, 5, 4, 11, 6, 4, 2, 9, 2, 8, 13, 12, 8]"
75,22,type-based,0.1,0.350000,0.750000,0.950000,0.950000,0.950000,"[10, 1, 4, 4, 5, 5, 148, 8, 12, 10, 2, 2, 1, 5..."
76,19,type-based,0.1,0.400000,0.600000,0.900000,0.900000,0.900000,"[1, 2, 3, 4, 161, 6, 11, 11, 10, 6]"
